# Activity 2.2 - Transfer Learning

**NAME:** Abo, Emmanuel Q.

**Section:** CPE32S8

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [2]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 15
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [3]:

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [4]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Create two datasets
* one with digits below 5
* one with 5 and above

In [6]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [7]:


feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [8]:


classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [9]:

model = Sequential(feature_layers + classification_layers)

Check the model summary

In [10]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [11]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/15
230/230 [==============================] - 47s 195ms/step - loss: 1.6083 - accuracy: 0.2220 - val_loss: 1.5793 - val_accuracy: 0.3069
Epoch 2/15
230/230 [==============================] - 42s 183ms/step - loss: 1.5760 - accuracy: 0.2816 - val_loss: 1.5415 - val_accuracy: 0.4102
Epoch 3/15
230/230 [==============================] - 43s 187ms/step - loss: 1.5421 - accuracy: 0.3529 - val_loss: 1.5002 - val_accuracy: 0.4960
Epoch 4/15
230/230 [==============================] - 44s 192ms/step - loss: 1.5022 - accuracy: 0.4278 - val_loss: 1.4535 - val_accuracy: 0.6106
Epoch 5/15
230/230 [==============================] - 42s 182ms/step - loss: 1.4596 - accuracy: 0.4937 - val_loss: 1.4015 - val_accuracy: 0.6881
Epoch 6/15
230/230 [==============================] - 41s 180ms/step - loss: 1.4126 - accuracy: 0.5487 - val_loss: 1.3434 - val_accuracy: 0.7367
Epoch 7/15
230/230 [==============================] - 41s 

Freeze only the feature layers

In [12]:
for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [14]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/15
240/240 [==============================] - 17s 70ms/step - loss: 1.5853 - accuracy: 0.3664 - val_loss: 1.4251 - val_accuracy: 0.4197
Epoch 2/15
240/240 [==============================] - 16s 69ms/step - loss: 1.4239 - accuracy: 0.4189 - val_loss: 1.2813 - val_accuracy: 0.4635
Epoch 3/15
240/240 [==============================] - 17s 70ms/step - loss: 1.2951 - accuracy: 0.4758 - val_loss: 1.1564 - val_accuracy: 0.5410
Epoch 4/15
240/240 [==============================] - 17s 69ms/step - loss: 1.1870 - accuracy: 0.5333 - val_loss: 1.0452 - val_accuracy: 0.6379
Epoch 5/15
240/240 [==============================] - 16s 65ms/step - loss: 1.0916 - accuracy: 0.5993 - val_loss: 0.9496 - val_accuracy: 0.7219
Epoch 6/15
240/240 [==============================] - 16s 67ms/step - loss: 1.0049 - accuracy: 0.6602 - val_loss: 0.8647 - val_accuracy: 0.7799
Epoch 7/15
240/240 [==============================] - 15s 64ms/s

#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [15]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/15
240/240 [==============================] - 17s 70ms/step - loss: 0.5493 - accuracy: 0.8633 - val_loss: 0.4290 - val_accuracy: 0.9148
Epoch 2/15
240/240 [==============================] - 17s 71ms/step - loss: 0.5244 - accuracy: 0.8710 - val_loss: 0.4075 - val_accuracy: 0.9181
Epoch 3/15
240/240 [==============================] - 17s 69ms/step - loss: 0.5027 - accuracy: 0.8761 - val_loss: 0.3882 - val_accuracy: 0.9222
Epoch 4/15
240/240 [==============================] - 17s 70ms/step - loss: 0.4823 - accuracy: 0.8825 - val_loss: 0.3716 - val_accuracy: 0.9264
Epoch 5/15
240/240 [==============================] - 19s 78ms/step - loss: 0.4651 - accuracy: 0.8848 - val_loss: 0.3561 - val_accuracy: 0.9301
Epoch 6/15
240/240 [==============================] - 15s 65ms/step - loss: 0.4502 - accuracy: 0.8905 - val_loss: 0.3422 - val_accuracy: 0.9313
Epoch 7/15
240/240 [==============================] - 16s 68ms/s

In [16]:
for l in feature_layers:
    l.trainable = False

In [17]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/15
230/230 [==============================] - 16s 66ms/step - loss: 1.5062 - accuracy: 0.4542 - val_loss: 1.3194 - val_accuracy: 0.5159
Epoch 2/15
230/230 [==============================] - 16s 70ms/step - loss: 1.3462 - accuracy: 0.4925 - val_loss: 1.1767 - val_accuracy: 0.5591
Epoch 3/15
230/230 [==============================] - 18s 78ms/step - loss: 1.2281 - accuracy: 0.5332 - val_loss: 1.0618 - val_accuracy: 0.6015
Epoch 4/15
230/230 [==============================] - 15s 65ms/step - loss: 1.1272 - accuracy: 0.5735 - val_loss: 0.9678 - val_accuracy: 0.6425
Epoch 5/15
230/230 [==============================] - 16s 69ms/step - loss: 1.0536 - accuracy: 0.6055 - val_loss: 0.8893 - val_accuracy: 0.6783
Epoch 6/15
230/230 [==============================] - 15s 63ms/step - loss: 0.9773 - accuracy: 0.6398 - val_loss: 0.8239 - val_accuracy: 0.7132
Epoch 7/15
230/230 [==============================] - 16s 70ms/s

#### Conclusion

#This activity has helped me understand how to apply transfer learning along with its benefits. It can be used to enhance the performance of the model by leveraging the knowledge gained from the pre-trained model. The process of applying transfer learning to build and train a neural network involves selecting a pre-trained model and freezing pre-trained layers to prevent their weights from being updated during training which can help prevent overfitting and reduce training time.

Colab Link: https://colab.research.google.com/drive/17otveCdQvKC0Hn9HAROYPPFW3T-NiaRL?usp=sharing